# Desafio Ciência de Dados

### João Victor Magno

### Primeiramente começamos importando as bibliotecas e funções que vamos utilizar no processo.

In [142]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import joblib

### Carregamos o dataset e verificamos as informações brutas dele antes de começarmos a limpeza e tratamento dos dados.

Percebemos que a maioria das colunas é do tipo object e existem algumas colunas com valores nulos que precisaremos tratar.

In [127]:
df = pd.read_csv('desafio_indicium_imdb.csv')

print("Informações gerais:")
df.info()

print("\nDados brutos:")
print(df.describe())

print("\nNulos por coluna:")
print(df.isnull().sum())

Informações gerais:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     999 non-null    int64  
 1   Series_Title   999 non-null    object 
 2   Released_Year  999 non-null    object 
 3   Certificate    898 non-null    object 
 4   Runtime        999 non-null    object 
 5   Genre          999 non-null    object 
 6   IMDB_Rating    999 non-null    float64
 7   Overview       999 non-null    object 
 8   Meta_score     842 non-null    float64
 9   Director       999 non-null    object 
 10  Star1          999 non-null    object 
 11  Star2          999 non-null    object 
 12  Star3          999 non-null    object 
 13  Star4          999 non-null    object 
 14  No_of_Votes    999 non-null    int64  
 15  Gross          830 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 125.0+ KB

Dados brutos:
   

### Conversão para númerico

Convertemos as colunas Runtime, Gross, Released_Year e Certificate para tipos númericos. Percebemos que um dos valores de Released_Year se tornou nulo, então preenchemos ele com a moda. E também preenchemos os nulos da coluna Certificate com a moda.

Após isso, realizamos One-Hot Encoding para as colunas de Genre e Certificate para que elas se tornem númericas. Como essas colunas não tem tantos valores diferentes podemos fazer isso sem muitos problemas.

In [128]:
df_copy = df.copy()

# Tratamento dos dados de object para numerico
df_copy['Runtime'] = df_copy['Runtime'].str.replace(' min', '').astype(int)
df_copy['Gross'] = pd.to_numeric(df_copy['Gross'].str.replace(',', '', regex=False), errors='coerce')
df_copy['Released_Year'] = pd.to_numeric(df_copy['Released_Year'], errors='coerce')
df_copy['Certificate'].fillna(df_copy['Certificate'].mode()[0], inplace=True)

print(df_copy.isnull().sum())
df_copy['Released_Year'].fillna(df_copy['Released_Year'].mode()[0], inplace=True)

# Gerando One-Hot Encoding para as colunas Genre e Certificate
genre_dummies = df_copy['Genre'].str.get_dummies(sep=', ')
certificate_dummies = pd.get_dummies(df_copy['Certificate'], prefix='Cert')

df_copy = pd.concat([df_copy, genre_dummies, certificate_dummies], axis=1)

df_copy.drop(columns=['Genre', 'Certificate'], inplace=True)


df_copy.info()

Unnamed: 0         0
Series_Title       0
Released_Year      1
Certificate        0
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 51 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     999 non-null    int64  
 1   Series_Title   999 non-null    object 
 2   Released_Year  999 non-null    float64
 3   Runtime        999 non-null    int64  
 4   IMDB_Rating    999 non-null    float64
 5   Overview       999 non-null    object 
 6   Meta_score     842 non-null    float64
 7   Director       999 non-null    object 
 8   Star1          999 non-null    object 
 9   Star2          999 non-null    object 
 10  Star3        

/tmp/ipython-input-3204374444.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['Certificate'].fillna(df_copy['Certificate'].mode()[0], inplace=True)
/tmp/ipython-input-3204374444.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method

### Correlação com o Faturamento e Meta Score

Analisamos a correlação do faturamento (coluna Gross) com as outras colunas, e também a correlação do Meta Score com as outras. Para entendermos qual a melhor maneiras de preencher os valores nulos dessas colunas.

In [129]:
df_numeric = df_copy.select_dtypes(include=np.number)

corr_matrix = df_numeric.corr()

# Calculo da correlação com o Gross
gross_correlations = corr_matrix['Gross'].sort_values(ascending=False)

gross_correlations = gross_correlations.drop('Gross')
print("Correlações das outras colunas com o faturamento:")
print(gross_correlations)
print()

# Calculo da correlação com o Meta Score
Meta_score_correlations = corr_matrix['Meta_score'].sort_values(ascending=False)

Meta_score_correlations = Meta_score_correlations.drop('Meta_score')
print("Correlações das outras colunas com o Meta score:")
print(Meta_score_correlations)

Correlações das outras colunas com o faturamento:
No_of_Votes      0.589527
Adventure        0.451594
Action           0.320713
Released_Year    0.234115
Sci-Fi           0.205171
Animation        0.161705
Runtime          0.140002
IMDB_Rating      0.099393
Fantasy          0.099352
Family           0.056102
Western         -0.018606
Comedy          -0.019715
Sport           -0.020099
History         -0.026716
Meta_score      -0.030480
Thriller        -0.033501
Horror          -0.042222
Biography       -0.042270
Musical         -0.057152
War             -0.059058
Music           -0.065199
Film-Noir       -0.066753
Unnamed: 0      -0.068809
Mystery         -0.081972
Romance         -0.097186
Crime           -0.127196
Drama           -0.319840
Name: Gross, dtype: float64

Correlações das outras colunas com o Meta score:
IMDB_Rating      0.271374
Film-Noir        0.146495
Romance          0.100393
Animation        0.078940
Western          0.064669
Horror           0.052746
War           

Percebemos que para o Faturamento a coluna que mais se correlaciona com ele é o Número de Votos (No_of_Votes), faz sentido pois filmes que geram mais engajamento são filmes blockbusters mais populares que tem alta bilheteria. Assim entendemos que o Número de Votos pode ser um bom previsor do faturamento.

E, percebemos que para o Meta Score não existe uma coluna que se tem uma correlação tão forte como o Faturamento e o Número de Votos, as duas maiores correlações são a Nota do IMDB de maneira positiva e de maneira negativa o Ano de Lançamento. De certa forma a correlação entre a nota do IMDB e o Meta Score é esperado, pois imaginamos que filmes com uma nota alta dos críticos também tenha uma nota alta para o público. Também podemos inferir que os filmes mais antigos tem uma maior nota da crítica, isso ocorre por conta de um fenômeno chamado viés da sobrevivência que explica que para um filme que foi lançado há muito tempo atrás estar em uma lista de melhores filmes de todos os tempos ele deve ser uma obra prima atemporal. Pois todos os filmes somente bons dessa época foram esquecidos com o tempo, e somente a elite "sobreviveu".

### Preenchendo os dados nulos

Como analisamos anteriormente, o Número de Votos tem uma forte correlação com o Faturamento. Dessa maneira, para preenchermos os dados nulos do Faturamento utilizaremos Machine Learning para treinar um modelo de Regressão Linear que recebe de entrada como os dados de teste a coluna de Número de Votos na qual o Faturamento não é nulo. E depois de treinarmos ele, utilizamos esse modelo para prever os dados de faturamento nulo usando de argumento de entrada o Número de Votos nas linhas nas quais o Faturamento é nulo.

In [130]:
df_test = df_copy.copy()

# Seleção dos dados de treinamento
train_data_Gross = df_test.dropna(subset=['Gross'])

X_train_impute_Gross = train_data_Gross[['No_of_Votes']]
y_train_impute_Gross = train_data_Gross['Gross']

# Treinamento do modelo
imputation_model_gross = LinearRegression()
imputation_model_gross.fit(X_train_impute_Gross, y_train_impute_Gross)


# Seleção dos dados que necessitam de previsão
data_to_predict_Gross = df_test[df_test['Gross'].isnull()]
X_to_predict_Gross = data_to_predict_Gross[['No_of_Votes']]

# Utilização do modelo
predicted_gross = imputation_model_gross.predict(X_to_predict_Gross)

# Preenchemos os valores nulos no dataframe com as previsões
df_test.loc[df_test['Gross'].isnull(), 'Gross'] = predicted_gross

print("Nulos por coluna depois da previsão:")
print(df_test.isnull().sum())

Nulos por coluna depois da previsão:
Unnamed: 0         0
Series_Title       0
Released_Year      0
Runtime            0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross              0
Action             0
Adventure          0
Animation          0
Biography          0
Comedy             0
Crime              0
Drama              0
Family             0
Fantasy            0
Film-Noir          0
History            0
Horror             0
Music              0
Musical            0
Mystery            0
Romance            0
Sci-Fi             0
Sport              0
Thriller           0
War                0
Western            0
Cert_16            0
Cert_A             0
Cert_Approved      0
Cert_G             0
Cert_GP            0
Cert_PG            0
Cert_PG-13         0
Cert_Passed        0
Cert_R             0
Cert_TV-14         0
Cert_TV-MA        

Agora fazemos o mesmo para o Meta Score. Treinamos o modelo de Regressão Linear com os argumentos de entrada as colunas de Ano de Lançamento e Nota do IMDB nas quais o Meta Score não é nulo. E após treinarmos utilizamos o modelo para prever os valores nulos do Meta Score e preenchemos eles.

In [131]:
# Seleção dos dados de treinamento
train_data_MS = df_copy.dropna(subset=['Meta_score', 'Released_Year', 'IMDB_Rating'])

X_train_impute_MS = train_data_MS[['Released_Year', 'IMDB_Rating']]
y_train_impute_MS = train_data_MS['Meta_score']

# Treinamento do modelo
imputation_model_MS = LinearRegression()
imputation_model_MS.fit(X_train_impute_MS, y_train_impute_MS)

# Seleção dos dados que necessitam de previsão
data_to_predict_MS = df_test[df_test['Meta_score'].isnull()]
X_to_predict_MS = data_to_predict_MS[['Released_Year', 'IMDB_Rating']]

# Utilização do modelo
predicted_metascore = imputation_model_MS.predict(X_to_predict_MS)


# Preenchemos os valores nulos no dataframe com as previsões
df_test.loc[X_to_predict_MS.index, 'Meta_score'] = predicted_metascore

print("Nulos por coluna depois da previsão:")
print(df_test.isnull().sum())

Nulos por coluna depois da previsão:
Unnamed: 0       0
Series_Title     0
Released_Year    0
Runtime          0
IMDB_Rating      0
Overview         0
Meta_score       0
Director         0
Star1            0
Star2            0
Star3            0
Star4            0
No_of_Votes      0
Gross            0
Action           0
Adventure        0
Animation        0
Biography        0
Comedy           0
Crime            0
Drama            0
Family           0
Fantasy          0
Film-Noir        0
History          0
Horror           0
Music            0
Musical          0
Mystery          0
Romance          0
Sci-Fi           0
Sport            0
Thriller         0
War              0
Western          0
Cert_16          0
Cert_A           0
Cert_Approved    0
Cert_G           0
Cert_GP          0
Cert_PG          0
Cert_PG-13       0
Cert_Passed      0
Cert_R           0
Cert_TV-14       0
Cert_TV-MA       0
Cert_TV-PG       0
Cert_U           0
Cert_U/A         0
Cert_UA          0
Cert_Unrated 

### Tratando as colunas Diretor e Estrelas 1 a 4

A ideia para tratar as colunas de Diretor e Estrelas, era realizar um one hot coding nelas e analisar a correlação de cada diretor e cada estrela com a nota IMDB para escolhermos somente as estrelas e os diretores que tenham uma alta correlação com a Nota IMDB do filme.

Porém após analisarmos os resultados, percebemos que praticamente eles nao têm uma influência grande na Nota IMDB com a maior de cada coluna sendo entre 0.13 e 0.19. Dessa maneira optamos por deixar somente o diretor e as estrelas com a maior correlação por coluna e removemos o resto do dataframe. Assim deixamos o Diretor com a maior correlação, a Estrela 1 com a maior correlação e assim em diante.

In [132]:
colunas = ['Director', 'Star1', 'Star2', 'Star3', 'Star4']

# Análise de quantidade de valores em cada coluna
print("Quantidade de Diretores:")
print(df['Director'].nunique())
print("Quantidade de Atores em Star1:")
print(df['Star1'].nunique())
print("Quantidade de Atores em Star2:")
print(df['Star2'].nunique())
print("Quantidade de Atores em Star3:")
print(df['Star3'].nunique())
print("Quantidade de Atores em Star4:")
print(df['Star4'].nunique())

colunas_mais_influentes = []

# Para cada coluna calculamos a correlação com a nota imdb
for coluna in colunas:
    print(f"\nCorrelação para: {coluna}")

    dummies = pd.get_dummies(df_copy[coluna], prefix=coluna)

    df_corr = pd.concat([dummies, df_copy['IMDB_Rating']], axis=1)

    correlations = df_corr.corr()['IMDB_Rating'].sort_values(ascending=False)

    correlations = correlations.drop('IMDB_Rating')

    print("\n10 maiores correlações positivas:")
    print(correlations.head(10))

    print("\n10 maiores correlações negativas:")
    print(correlations.tail(10))

    # Retiramos as 5 colunas com as maiores correlações em módulo
    top_5_features = correlations.abs().nlargest(5).index.tolist()
    colunas_mais_influentes.extend(top_5_features)

Quantidade de Diretores:
548
Quantidade de Atores em Star1:
659
Quantidade de Atores em Star2:
840
Quantidade de Atores em Star3:
890
Quantidade de Atores em Star4:
938

Correlação para: Director

10 maiores correlações positivas:
Director_Christopher Nolan       0.169873
Director_Peter Jackson           0.117806
Director_Francis Ford Coppola    0.117806
Director_Charles Chaplin         0.110074
Director_Akira Kurosawa          0.100517
Director_Stanley Kubrick         0.099982
Director_Sergio Leone            0.091032
Director_Milos Forman            0.090852
Director_Lana Wachowski          0.087472
Director_Irvin Kershner          0.087472
Name: IMDB_Rating, dtype: float64

10 maiores correlações negativas:
Director_Shane Meadows         -0.049034
Director_Brian G. Hutton       -0.049034
Director_Jean-Jacques Annaud   -0.049034
Director_Spike Lee             -0.051941
Director_Woody Allen           -0.055725
Director_Alfonso Cuarón        -0.056538
Director_Don Siegel            -0.

Adicionamos as colunas que verificamos no passo anterior serem as com as maiores correlações com o IMDB e retiramos as colunas de Diretor, as de Estrelas, a de Título e a de Sinopse. Também retiramos essa coluna chamada Unnamed: 0 que é somente a coluna dos indices do csv.

In [133]:
for coluna in colunas_mais_influentes:
    # Obter o nome original da coluna e o o valor dela
    coluna_original, valor = coluna.split('_', 1)
    # Verificação para preencher corretamente se o diretor ou a estrela estiverem no filme
    df_test[coluna] = (df_test[coluna_original] == valor).astype(int)

df_test.drop(columns = ['Director', 'Star1', 'Star2', 'Star3', 'Star4', 'Series_Title', 'Overview', 'Unnamed: 0'], inplace=True)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 68 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Released_Year                  999 non-null    float64
 1   Runtime                        999 non-null    int64  
 2   IMDB_Rating                    999 non-null    float64
 3   Meta_score                     999 non-null    float64
 4   No_of_Votes                    999 non-null    int64  
 5   Gross                          999 non-null    float64
 6   Action                         999 non-null    int64  
 7   Adventure                      999 non-null    int64  
 8   Animation                      999 non-null    int64  
 9   Biography                      999 non-null    int64  
 10  Comedy                         999 non-null    int64  
 11  Crime                          999 non-null    int64  
 12  Drama                          999 non-null    int

### Normalização e análise de correlação com o IMDB

Primeiramente analisamos a correlação de cada uma das nossas colunas com a Nota IMDB para entendermos se podemos retirar algumas colunas que não tem tanta influência na nota. Para isso pensamos em testar 3 escalas e comparar o resultado do modelo para cada uma dessas escalas.

- Primeira escala: retirar as colunas com correlação menores em módulo que 0.01
- Segunda escala: retirar as colunas com correlação menores em módulo que 0.05
- Terceira escala: retirar as colunas com correlação menores em módulo que 0.1

Outra coisa que podemos perceber ao analisar essas correlações é que a maior correlação com a nota IMDB é o número de votos, seguido pela nota da crítica seguido pela duração do filme. Isso nos mostra que quanto maior o engajamento que o filme esta recebendo de votos maior a sua nota. Podemos também dizer que filmes de alta qualidade são aclamados tanto pelos críticos quanto pelo público.
E também que filmes de maior duração tendem a ter notas maiores, assim indicando que histórias mais complexas que exigem maior tempo de tela tendem a ser mais bem avaliadas.

Analisando pelo lado negativo, podemos voltar a citar o viés de sobreviência dos filmes, por conta do ano de lançamento ser o fator que mais influência negativamente a nota desse filme. Podemos citar que as classificações indicativas R e PG-13 afetam a nota imdb do filme negativamente também, assim filmes com essas classificações tendem a não ter uma nota mais alta.

In [134]:
corr_matrix_imdb = df_test.corr()
imdb_correlations = corr_matrix_imdb['IMDB_Rating'].sort_values(ascending=False)

imdb_correlations = imdb_correlations.drop('IMDB_Rating')

print("Correlações com a Nota IMDB:")
print(imdb_correlations)

# Retirando as colunas com baixa correlação de acordo com a escala
colunas_ruins_escala1 = imdb_correlations[(imdb_correlations > -0.01) & (imdb_correlations < 0.01)].index.tolist()
colunas_ruins_escala2 = imdb_correlations[(imdb_correlations > -0.05) & (imdb_correlations < 0.05)].index.tolist()
colunas_ruins_escala3 = imdb_correlations[(imdb_correlations > -0.1) & (imdb_correlations < 0.1)].index.tolist()
df_normalizado1 = df_test.drop(columns = colunas_ruins_escala1)
df_normalizado2 = df_test.drop(columns = colunas_ruins_escala2)
df_normalizado3 = df_test.drop(columns = colunas_ruins_escala3)
print("\n")
print("Colunas retiradas na primeira escala:")
print(colunas_ruins_escala1)
print("Colunas retiradas na segunda escala:")
print(colunas_ruins_escala2)
print("Colunas retiradas na terceira escala:")
print(colunas_ruins_escala3)

# Normalizar as colunas necessárias
colunas_para_normalizar_1 = df_normalizado1.select_dtypes(include=np.number).columns.tolist()
colunas_para_normalizar_2 = df_normalizado2.select_dtypes(include=np.number).columns.tolist()
colunas_para_normalizar_3 = df_normalizado3.select_dtypes(include=np.number).columns.tolist()

colunas_para_normalizar_1.remove('IMDB_Rating')
colunas_para_normalizar_2.remove('IMDB_Rating')
colunas_para_normalizar_3.remove('IMDB_Rating')

scaler_1 = MinMaxScaler()
scaler_2 = MinMaxScaler()
scaler_3 = MinMaxScaler()

df_normalizado1[colunas_para_normalizar_1] = scaler_1.fit_transform(df_normalizado1[colunas_para_normalizar_1])
df_normalizado2[colunas_para_normalizar_2] = scaler_2.fit_transform(df_normalizado2[colunas_para_normalizar_2])
df_normalizado3[colunas_para_normalizar_3] = scaler_3.fit_transform(df_normalizado3[colunas_para_normalizar_3])

Correlações com a Nota IMDB:
No_of_Votes           0.479308
Meta_score            0.283265
Runtime               0.242751
Star4_Diane Keaton    0.189595
Star1_Elijah Wood     0.171824
                        ...   
Thriller             -0.056337
Comedy               -0.090209
Cert_PG-13           -0.117105
Cert_R               -0.118701
Released_Year        -0.134336
Name: IMDB_Rating, Length: 67, dtype: float64


Colunas retiradas na primeira escala:
['Adventure', 'History', 'Action', 'Musical', 'Cert_Approved', 'Cert_TV-PG']
Colunas retiradas na segunda escala:
['Cert_TV-14', 'Cert_U', 'Sci-Fi', 'Western', 'Mystery', 'Film-Noir', 'Cert_G', 'Cert_16', 'Cert_TV-MA', 'Cert_Unrated', 'Cert_UA', 'Crime', 'Adventure', 'History', 'Action', 'Musical', 'Cert_Approved', 'Cert_TV-PG', 'Sport', 'Cert_PG', 'Biography', 'Fantasy', 'Cert_GP', 'Animation', 'Music', 'Romance', 'Family', 'Horror', 'Cert_U/A']
Colunas retiradas na terceira escala:
['Gross', 'Cert_A', 'Drama', 'War', 'Cert_Passed', 'Cer

### Treinando os Modelos

Agora que temos nosso dataframe normalizado e com os dados todos tratados, vamos treinar os modelos utilizando a regressao por meio do algorítmo de Random Forest que como a nota IMDB de um filme é um fenômeno complexo ele se sai melhor que a regressão linear nesses casos. E também por conta que ele faz um bom papel em captar a interação entre features.

Assim treinamos 3 modelos, um para cada dataframe que temos.

In [135]:
X_1 = df_normalizado1.drop(columns=['IMDB_Rating'])
y_1 = df_normalizado1['IMDB_Rating']
final_feature_columns_1 = X_1.columns.tolist()

# Divisão em dados de treino e dados de teste
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.2, random_state=42)

# Treinamento do modelo
model_1 = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_1.fit(X_train_1, y_train_1)

# Utilização do modelo
y_pred_1 = model_1.predict(X_test_1)

# Métricas de desempenho
mae_1 = mean_absolute_error(y_test_1, y_pred_1)
mse_1 = mean_squared_error(y_test_1, y_pred_1)
r2_1 = r2_score(y_test_1, y_pred_1)

print("--- Resultados da Avaliação do Modelo Final (Random Forest) ---")
print(f"Mean Absolute Error (MAE): {mae_1:.4f}")
print(f"Mean Squared Error (MSE): {mse_1:.4f}")
print(f"R-squared (R²): {r2_1:.4f}")

--- Resultados da Avaliação do Modelo Final (Random Forest) ---
Mean Absolute Error (MAE): 0.1505
Mean Squared Error (MSE): 0.0369
R-squared (R²): 0.4385


O segundo modelo

In [136]:
X_2 = df_normalizado2.drop(columns=['IMDB_Rating'])
y_2 = df_normalizado2['IMDB_Rating']
final_feature_columns_2 = X_2.columns.tolist()

# Divisão em dados de treino e dados de teste
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=42)

# Treinamento do modelo
model_2 = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_2.fit(X_train_2, y_train_2)

# Utilização do modelo
y_pred_2 = model_2.predict(X_test_2)

# Métricas de desempenho
mae_2 = mean_absolute_error(y_test_2, y_pred_2)
mse_2 = mean_squared_error(y_test_2, y_pred_2)
r2_2 = r2_score(y_test_2, y_pred_2)

print("--- Resultados da Avaliação do Modelo Final (Random Forest) ---")
print(f"Mean Absolute Error (MAE): {mae_2:.4f}")
print(f"Mean Squared Error (MSE): {mse_2:.4f}")
print(f"R-squared (R²): {r2_2:.4f}")

--- Resultados da Avaliação do Modelo Final (Random Forest) ---
Mean Absolute Error (MAE): 0.1540
Mean Squared Error (MSE): 0.0381
R-squared (R²): 0.4191


O terceiro modelo

In [137]:
X_3 = df_normalizado3.drop(columns=['IMDB_Rating'])
y_3 = df_normalizado3['IMDB_Rating']
final_feature_columns_3 = X_3.columns.tolist()

# Divisão em dados de treino e dados de teste
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y_3, test_size=0.2, random_state=42)

# Treinamento do modelo
model_3 = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_3.fit(X_train_3, y_train_3)

# Utilização do modelo
y_pred_3 = model_3.predict(X_test_3)

# Métricas de desempenho
mae_3 = mean_absolute_error(y_test_3, y_pred_3)
mse_3 = mean_squared_error(y_test_3, y_pred_3)
r2_3 = r2_score(y_test_3, y_pred_3)

print("Resultados Modelo 3:")
print(f"Erro absoluto médio: {mae_1:.4f}")
print(f"Erro quadrático médio: {mse_1:.4f}")
print(f"R-quadrado: {r2_1:.4f}")

Resultados Modelo 3:
Erro absoluto médio: 0.1505
Erro quadrático médio: 0.0369
R-quadrado: 0.4385


### Análise de resultados dos modelos

Podemos perceber que todos tem bons resultados, com uma taxa de erro absoluto menor que 0.2. Porém percebemos que o modelo originado pela escala 2 é um pouco menos preciso que os outros dois. E também podemos inferir que o modelo 1 e o modelo 3 são praticamente iguais no quesito performance, tendo os mesmos resultados nas métricas de desempenho. Dessa maneira escolheremos o modelo 3 por ter um número bem menor de dimensões e mantendo a precisão do algorítmo.

In [146]:
# Salvando o modelo em arquivo .pkl
joblib.dump(model_3, "modelo_imdb.pkl")

['modelo_imdb.pkl']

### Preparando o novo filme para a previsão

Para utilizarmos o modelo nesse novo filme, temos que preparar ele para os moldes que o modelo foi treinado. Para isso ele passa pela mesma pipeline de tratamento e limpeza de dados feita no dataframe original.

In [138]:
novo_filme = {'Series_Title': 'The Shawshank Redemption',
 'Released_Year': '1994',
 'Certificate': 'A',
 'Runtime': '142 min',
 'Genre': 'Drama',
 'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'Meta_score': 80.0,
 'Director': 'Frank Darabont',
 'Star1': 'Tim Robbins',
 'Star2': 'Morgan Freeman',
 'Star3': 'Bob Gunton',
 'Star4': 'William Sadler',
 'No_of_Votes': 2343110,
 'Gross': '28,341,469'}

novo_filme_df = pd.DataFrame([novo_filme])

# Aplicar o mesmo pipeline de tratamento e limpeza
novo_filme_df['Runtime'] = novo_filme_df['Runtime'].str.replace(' min', '').astype(int)
novo_filme_df['Gross'] = pd.to_numeric(novo_filme_df['Gross'].str.replace(',', '', regex=False), errors='coerce')
novo_filme_df['Released_Year'] = pd.to_numeric(novo_filme_df['Released_Year'], errors='coerce')
novo_filme_df['Certificate'].fillna(novo_filme_df['Certificate'].mode()[0], inplace=True)
novo_filme_df['Released_Year'].fillna(novo_filme_df['Released_Year'].mode()[0], inplace=True)

genre_dummies_new = novo_filme_df['Genre'].str.get_dummies(sep=', ')
certificate_dummies_new = pd.get_dummies(novo_filme_df['Certificate'], prefix='Cert')
director_dummies_new = pd.get_dummies(novo_filme_df['Director'], prefix='Director')
star1_dummies_new = pd.get_dummies(novo_filme_df['Star1'], prefix='Star1')
star2_dummies_new = pd.get_dummies(novo_filme_df['Star2'], prefix='Star2')
star3_dummies_new = pd.get_dummies(novo_filme_df['Star3'], prefix='Star3')
star4_dummies_new = pd.get_dummies(novo_filme_df['Star4'], prefix='Star4')
novo_filme_df = pd.concat([novo_filme_df, genre_dummies_new, certificate_dummies_new, director_dummies_new, star1_dummies_new, star2_dummies_new, star3_dummies_new, star4_dummies_new], axis=1)

novo_filme_df.drop(columns=['Genre', 'Certificate', 'Director', 'Star1', 'Star2', 'Star3', 'Star4'], inplace=True)

# Alinhar as colunas do filme com as do modelo
novo_filme_processed_3 = novo_filme_df.reindex(columns=final_feature_columns_3, fill_value=0)

# Normalizar o filme
novo_filme_processed_3[colunas_para_normalizar_3] = scaler_3.transform(novo_filme_processed_3[colunas_para_normalizar_3])

/tmp/ipython-input-3235115172.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  novo_filme_df['Certificate'].fillna(novo_filme_df['Certificate'].mode()[0], inplace=True)
/tmp/ipython-input-3235115172.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'd

### Utilização do Modelo

Agora utilizamos o modelo para prever a Nota IMDB do filme Shawshank Redemption.

In [139]:
previsao = model_3.predict(novo_filme_processed_3)

print(f"\nNota IMDB prevista: {previsao[0]:.2f}")


Nota IMDB prevista: 8.78


### Análise dos Resultados

A nota prevista foi de 8.78, e a nota real desse filme é de 9.3. Assim o modelo teve um erro bruto de 0.52, que é maior que a média de erro do modelo que foi de 0.15. Isso é normal de acontecer pois esse é o filme mais bem ranqueado do IMDB, então seria difícil para o modelo ranquear ele da mesma maneira. Porém o modelo previu uma nota bem alta para o filme o ranqueando na 13º posição do database, assim entendemos que o modelo conseguiu identificar que o filme tem altíssima qualidade.

### Pergunta: Qual filme você recomendaria para uma pessoa que você não conhece?

Resposta: Eu recomendaria os filmes melhor ranqueados pela nota do imdb, pois como a nota do imdb é uma nota feita pelo público. Se eu nao tenho nenhum conhecimento prévio da pessoa a escolha segura é recomendar os filmes que tenham as maiores notas.

Os filmes com as 5 maiores notas baseado no database fornecido:
1. The Godfather
2. The Dark Night
3. The Lord of The rings
4. 12 Angry Men
5. The Lord of the Rings: The Return of the King

Porem eu definitivamente recomendaria também Shawshank Redemption, que é a maior nota do IMDB.

In [140]:
print(df['Series_Title'].head(5))

0                                    The Godfather
1                                  The Dark Knight
2                           The Godfather: Part II
3                                     12 Angry Men
4    The Lord of the Rings: The Return of the King
Name: Series_Title, dtype: object


### Pergunta: Quais são os principais fatores que estão relacionados com alta expectativa de faturamento de um filme?

Resposta: Como analisado durante o processo, os fatores que mais inluenciam no faturamento positivamente são: o número de votos, se o filme é do gênero ação e aventura. E os que mais influenciam negativamente no faturamento é se o filme é do gênero drama.

Isso se analisa que quanto mais engajamento o filme tem, mais gera a vontade das pessoas de irem aos cinamas assistir o filme, assim gerando mais faturamento. Também podemos inferir que o público que vai ao cinema tem uma clara preferência por filmes de ação e aventura e uma certa aversão a filmes de drama.

In [141]:
print(gross_correlations)

No_of_Votes      0.589527
Adventure        0.451594
Action           0.320713
Released_Year    0.234115
Sci-Fi           0.205171
Animation        0.161705
Runtime          0.140002
IMDB_Rating      0.099393
Fantasy          0.099352
Family           0.056102
Western         -0.018606
Comedy          -0.019715
Sport           -0.020099
History         -0.026716
Meta_score      -0.030480
Thriller        -0.033501
Horror          -0.042222
Biography       -0.042270
Musical         -0.057152
War             -0.059058
Music           -0.065199
Film-Noir       -0.066753
Unnamed: 0      -0.068809
Mystery         -0.081972
Romance         -0.097186
Crime           -0.127196
Drama           -0.319840
Name: Gross, dtype: float64


### Pergunta: Quais insights podem ser tirados com a coluna Overview? É possível inferir o gênero do filme a partir dessa coluna?

Resposta: A coluna Overview contém dados de texto não estruturados. Usando técnicas de Processamento de Linguagem Natural, podemos extrair insights que não são aparentes em colunas numéricas ou categóricas. Podemos sim inferir o gênero do filme a partir da coluna Overview, para realizar isso temos que ensinar um modelo a reconhecer os padrões de palavras presentes em cada gênero.